In [ ]:
# uses first sheet of scb excel

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pymc3 as pm
import arviz as az
import scipy.stats as sps
import requests

sns.set()

In [ ]:
def parse_scb_prel():
    
    def monthname2number(m):
        if m == 'januari' : return 1
        if m == 'februari' : return 2
        if m == 'mars' : return 3
        if m == 'april' : return 4
        if m == 'maj' : return 5
        if m == 'juni' : return 6
        if m == 'juli' : return 7
        if m == 'augusti' : return 8
        if m == 'september' : return 9
        if m == 'oktober' : return 10
        if m == 'november' : return 11
        if m == 'december' : return 12

    df = pd.read_csv('scb-dead_per_day.csv',sep=';',usecols=[1,2,3,4,5,6,7,10,11],thousands=',')
    
    df['month'] = df['Månad'].apply(monthname2number)
    df[['Månad','month']] = df[['Månad','month']].replace(np.nan,0)
    df['month'] = df['month'].astype(int)
    
    df[['2015','2016','2017','2018','2019','2020','2015-2019']] = df[['2015','2016','2017','2018','2019','2020','2015-2019']].astype(int)
    
    unknown_death_day = df.iloc[-1,:7]
    
    df.drop(366,inplace=True)
    
    df = df.loc[df['2020'] != 0]
    df = df.drop('Månad',axis=1)
    
    known_death_day = df.iloc[:,:7].sum()
    
    year_fraction = len(df) / 366
    
    unknown_death_day[['2015','2016','2017','2018','2019','2015-2019']] = unknown_death_day[
    ['2015','2016','2017','2018','2019','2015-2019']] * year_fraction
    
    dead_pct_per_day = df[['2015','2016','2017','2018','2019','2020','2015-2019']] / known_death_day
    
    all_deaths = df[['2015','2016','2017','2018','2019','2020','2015-2019']] + unknown_death_day * dead_pct_per_day
    
    index = pd.date_range('2020-01-01',periods=len(all_deaths))
    all_deaths.index = index
    
    return all_deaths
    
    

In [ ]:
# days with k homicides in UK
zeros = np.array([0] * 259)
ones = np.array([1] * 387)
twos = np.array([2] * 261)
threes = np.array([3] * 131)
fours = np.array([4] * 40)
fives = np.array([5] * 13)
six = np.array([6] * 3)

data = np.concatenate([zeros,ones,twos,threes,fours,fives,six])


In [ ]:
data_mean = data.mean()
data_std = data.std()
print (data_mean)
print (data_std)

In [ ]:
scb = parse_scb_prel()
scb

In [ ]:
scb.loc[:'2020-06-30'].sum()

In [ ]:
 # första halvåret
scb.loc[:'2020-06-30'].sum().loc['2020'] - scb.loc[:'2020-06-30'].sum().loc['2015-2019']

In [ ]:
last_data_day = scb.index[-1]
last_data_day

In [ ]:
day_excess = scb['2020'][:last_data_day] - scb['2015-2019'][:last_data_day]
idx = day_excess.index.date
day_excess.plot(x=idx,figsize=(18,12),style='ro--',
               title='Sweden Daily Excess Deaths YTD {}\nDataSource:SCB.se'.format(last_data_day.date()))
plt.ylabel('Excess Deaths per day')

plt.savefig('scb_excess_deaths_per_day.jpg',format='jpg')

In [ ]:
cumulative_until_last_day = scb[:last_data_day].sum()
cumulative_until_last_day['2020'] / cumulative_until_last_day['2015-2019']

In [ ]:
scb_weekly = scb.resample('W-TUE').sum()
print(scb_weekly.loc[:last_data_day].sum())
(scb_weekly.loc[:last_data_day,'2020'] - scb_weekly.loc[:last_data_day,'2015-2019']).sum()

In [ ]:
scb_weekly

In [ ]:
scb_weekly.loc[:'2020-07-31'].sum()

In [ ]:
scb_weekly['excess'] = scb_weekly['2020'] - scb_weekly['2015-2019']
scb_weekly['cumexcess'] = scb_weekly['excess'].cumsum()
scb_weekly

In [ ]:
scb_monthly = scb.resample('M').sum()
scb_monthly['delta'] = scb_monthly['2020'] - scb_monthly['2015-2019']
scb_monthly['cumdelta'] = (scb_monthly['delta'].cumsum())
# last of feb not correct for delta - leap year
scb_monthly

In [ ]:
scb_monthly.loc[:'2020-07-31'].sum()

In [ ]:
scb_monthly.loc[:'2020-07-31'].sum()['2020'] / 10.327589

In [ ]:
data = scb_weekly.iloc[:,0:5]
#data = data.iloc[:9]
data.mean(axis=1)
data

In [ ]:
data_min = data.min(axis=1)
data_max = data.max(axis=1)

print (data.mean().mean())

In [ ]:
walk = 1000
tune = 500
chains = 2


res_array = np.zeros((len(data),3))
dist_array = np.zeros((len(data),walk*chains))

for i in range(len(data)):
    model = pm.Model()
    with model:
        print ('\n++++++++++++++++++++ processing day ++++++++++++++++++++', i)
        print (data.iloc[i,:])
        print (data.iloc[i,:].mean())

        lambda_ = pm.Exponential('lambda',1/ data.iloc[i,:].mean())
        lkh  = pm.Poisson('lkh',lambda_,observed=data.iloc[i,:])
        
        trace = pm.sample(walk,tune=tune)
        post = trace['lambda'][:]   

        res_array[i,0] = post.min()
        res_array[i,1] = post.mean()
        res_array[i,2] = post.max()

        dist_array[i,:] = post
                     
    

In [ ]:
dist_array

In [ ]:
N = 1000

lambdas_per_week = np.zeros((len(dist_array),N))
samples_per_week = np.zeros((len(dist_array),N))

for week in range(len(dist_array)):
    lambdas_per_week[week] = np.random.choice(dist_array[week],replace=True,size=N)
    pois_dist = pm.Poisson.dist(lambdas_per_week[week])
    samples_per_week[week] = pois_dist.random(size=N)

In [ ]:
print (lambdas_per_week[:5])
samples_per_week[:5]

In [ ]:
ci_samples = np.percentile(samples_per_week,[5.5,94.5],axis=1)
ci_samples

plt.fill_between(scb_weekly.index,ci_samples[0],ci_samples[1])

In [ ]:
res_array

In [ ]:
errors_low = np.abs(res_array[:,1] - res_array[:,0])
errors_low = errors_low[:-1]
errors_high = np.abs(res_array[:,1] - res_array[:,2]) 
errors_high = errors_high[:-1]
print (errors_low)
print (errors_high)

errors = np.array((errors_low,errors_high))
errors

In [ ]:
excess = scb_weekly.loc[:last_data_day,'2020'] - scb_weekly.loc[:last_data_day,'2015-2019']
print (excess)
current_excess = excess.cumsum()
print (current_excess)
excess.index = excess.index.date
excess.plot.bar(x=excess.index,title='Sweden Weekly Excess deaths YTD {}\nDataSource: SCB.se'.format(last_data_day.date()),
                y=excess,
                figsize=(18,12),
                color='crimson')
plt.ylabel('Number of Excess Deaths')
plt.savefig('swe_excess_deaths.jpg',format='jpg',dpi=400)

In [ ]:
plt.figure(figsize=(18,12))
plt.title('Sweden weekly & excess deaths 2020 Y2D '\
          '{} compared to average 2015-2019 Current nr Excess Deaths: {}\n$Data Source: SCB.se$'.format(
    excess.index[-1],int(current_excess[-1])))
ax = plt.gca()
ax2 = plt.twinx()

ax.errorbar(scb_weekly.index[:len(data) - 1],res_array[:-1,1],yerr=errors,
             fmt='o',capsize=5,color='lime',alpha=0.6,label='Posterior sample means with CI')

ax.plot(scb_weekly.index[:len(data) -1],scb_weekly['2015-2019'][:-1],'o--',
        markerfacecolor='none',markeredgecolor='navy',markersize=12,
        label='2015-2019 mean')

ax.plot(scb_weekly[:last_data_day].index,scb_weekly.loc[:last_data_day,'2020'],'o--',
         color='crimson',label='2020 data')

ax.fill_between(scb_weekly.index[:len(data) -1],y1=data_min[:-1],y2=data_max[:-1],color='grey',
                 alpha=0.2,label='2015-2019 range')

ax.fill_between(scb_weekly.index[:len(data) -1],y1=ci_samples[0][:-1],y2=ci_samples[1][:-1],color='c',
                 alpha=0.2,label='Poisson Sample 89% CI')



ax2.plot(excess.index,excess.cumsum(),'o--',ls='dashed',color='orange',label='Cumulative Excess Deaths')
ax.set_ylabel('number of weekly deaths')
ax2.set_ylabel('cumulative Excess deaths')
ax.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.savefig('scb_poisson_weekly.jpg',format='jpg',dpi=400)

In [ ]:
#scb_monthly.index = scb_monthly.index.date
scb_monthly[['2020','2015-2019']][:pd.Timestamp('2020-07-31')].plot.bar(color=['r','orange'],
                                                                        title='Sweden deaths per month\nDataSource: SCB.se',figsize=(18,12))
plt.ylabel('Number of deaths')
plt.xticks(range(len(scb_monthly[:pd.Timestamp('2020-07-31')])),['Jan','Feb','Mar','Apr','May','Jun','Jul'])
plt.savefig('scb_monthly_deaths.jpg',format='jpg')

In [ ]:
### population 2014-2019
url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101A/BefolkningR1860'
data = {
  "query": [
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:ÅlderTotA",
        "values": []
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2014",  
          "2015",
          "2016",
          "2017",
          "2018",
          "2019"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r = requests.post(url,json=data)
r.status_code

In [ ]:
json = r.json()

In [ ]:
data_list = []

for rec in range(len(json['data'])):
    data_list.append((json['data'][rec]['key'][0],json['data'][rec]['key'][1],json['data'][rec]['values'][0]))

In [ ]:
population = pd.DataFrame(data_list,columns=['gender','year','population'])
population = population.set_index('year')
population['population'] = population['population'].astype(int)

In [ ]:
population = population.groupby(population.index).sum()
population['growth'] = population['population'] / population['population'].shift()


population.loc['2020','population'] = population.loc['2019','population']  
population['population'] = population['population'].astype(int)
population

In [ ]:
pop = population['population']
pop = pop.shift()
pop

In [ ]:
pop.drop('2014',inplace=True)
pop

In [ ]:
weekly_deaths = scb_weekly.loc[:,'2015':'2020'].copy()
weekly_deaths['2015-2018'] = weekly_deaths.loc[:,'2015':'2018'].mean(axis=1)
weekly_deaths = weekly_deaths[:-1]
weekly_deaths

In [ ]:
weekly_deaths_per_m = weekly_deaths / (pop / 1e6)
weekly_deaths_per_m['2015-2018'] = weekly_deaths_per_m.loc[:,'2015' : '2018'].mean(axis=1)
weekly_deaths_per_m['excess'] = weekly_deaths_per_m['2020'] - weekly_deaths_per_m['2015-2018']
weekly_deaths_per_m['cum_excess'] = weekly_deaths_per_m['excess'].cumsum()
weekly_deaths_per_m['abs_excess'] = weekly_deaths_per_m['cum_excess'] * (pop['2020'] / 1e6)
weekly_deaths_per_m['min'] = weekly_deaths_per_m.loc[:,'2015':'2018'].min(axis=1)
weekly_deaths_per_m['max'] = weekly_deaths_per_m.loc[:,'2015' :'2018'].max(axis=1)
weekly_deaths_per_m

In [ ]:
data = weekly_deaths_per_m.loc[:,'2015' : '2020'].copy()
data.drop('2015-2018',axis=1,inplace=True)
data

In [ ]:
data = data.unstack().reset_index()
data.columns=['year','week','dead_per_m']


In [ ]:
data['week']

In [ ]:
unique_weeks = data['week'].dt.date.unique()
week_idx_map = dict(zip(unique_weeks,range(len(unique_weeks))))
week_idx_map

In [ ]:
data['week_idx'] = data['week'].apply(lambda x : week_idx_map[pd.to_datetime(x).date()])
data.head(50)

In [ ]:
model = pm.Model()
week_idx = data['week_idx'].values

with model:

    lambda_bar = pm.Exponential('lambda_bar',1 / data['dead_per_m'].mean())
    lambda_ = pm.Exponential('lambda_',lambda_bar,shape=len(week_idx_map))
    lkh = pm.Poisson('lkh',lambda_[week_idx],observed=data['dead_per_m'])
    
    trace = pm.sample(500,tune=500)
    
    result = pm.trace_to_dataframe(trace)
    summary = az.summary(trace)
    az.plot_trace(trace)

In [ ]:
print (summary)

In [ ]:
result.describe()

In [ ]:
weekly_means = result.describe().loc['mean','lambda___0' :]

weekly_means

In [ ]:
CI = result.loc[:,'lambda___0':].quantile([0.055,0.945])
CI

In [ ]:
abs_error = np.abs(weekly_means - CI)
abs_error

In [ ]:
plt.figure(figsize=(18,12))

ax = plt.gca()

ax2 = plt.twinx()

ax.plot(weekly_deaths_per_m.index,weekly_deaths_per_m['2020'],'o--',
         color='crimson',label='weekly deaths per million')


ax.plot(weekly_deaths_per_m.index,weekly_deaths_per_m['2015-2018'],'o--',
         markerfacecolor='none',markeredgecolor='navy',label='average 2015-2018')

ax.fill_between(weekly_deaths_per_m.index,weekly_deaths_per_m['min'],weekly_deaths_per_m['max'],
                color='c',alpha=0.3,label='2015-2018 range')

ax.errorbar(weekly_deaths_per_m.index,weekly_means,yerr=abs_error.values,
            fmt='o--',capsize=5,mec='k',mfc='none',color='grey',label='Poisson expectation 89% CI')


ax.legend(loc='upper left')

ax2.plot(weekly_deaths_per_m.index,weekly_deaths_per_m['cum_excess'],'o--',color='orange',
        label='cumulative excess deaths per M')

plt.title('SWEDEN Weekly All Cause deaths & excess deaths, per million, cmp Baseline 2015-2018')
ax.set_ylabel('deaths per million')
ax2.set_ylabel('cumulative excess deaths per M')
ax2.legend(loc='upper right')

plt.savefig('scb_poisson_per_m.jpg',format='jpg')